In [3]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim           
import folium 
from folium import Marker
from folium.plugins import MarkerCluster


Initially I create a DataFrame <b>starbucks</b> containing Starbucks locations in the state of California.

In [4]:
starbucks = pd.read_csv("starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


Most of the stores have known (latitude, longitude) locations. But, all of the locations(they are five) in the city of Berkeley are missing.

In [5]:
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


So I'll fill in these values with the Nominatim geocoder. It takes the address name of a store and converts it into a geolocation.


In [6]:
geolocator = Nominatim(user_agent="Palina")

#A function that creates a geopy.point.Point object and returns a Series with its x,y-coordinates in Latitude and Longitude columns
def coord(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

#Updating 'starbucks' with created higher Series, where x - is every row of 'rows_with_missing'
starbucks.update(rows_with_missing.apply(lambda x: coord(x['Address']), axis=1))


After that I visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. I add a marker to each row of 'starbucks'

In [7]:
map = folium.Map(location=[37.88,-122.26], zoom_start=13)

for idx, row in starbucks[starbucks['City']=='Berkeley'].iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(map)

map

Then I load a GeoDataFrame <b>Calif_counties</b> containing the name, area (in square kilometers), and a unique id for each county in the state of California. The 'geometry' column contains a polygon with county boundaries.

After that I load 3 GeoDataFrames: <b>Calif_pop</b> with estimate of the population of each county, <b>Calif_high_earners</b> with the number of households with an income of at least $150,000 per year, <b>Calif_median_age</b> with the median age for each county.

In [12]:
Calif_counties = gpd.read_file("CA_county_boundaries/CA_county_boundaries.shp")
Calif_counties.crs = {'init': 'epsg:4326'}

Calif_pop = pd.read_csv("CA_county_population.csv", index_col="GEOID")
Calif_high_earners = pd.read_csv("CA_county_high_earners.csv", index_col="GEOID")
Calif_median_age = pd.read_csv("CA_county_median_age.csv", index_col="GEOID")

Calif_counties.head()

c:\Users\polin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.6556 39.69357, -120.65554 39.69..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


I want to join the <b>Calif_counties</b> GeoDataFrame with <b>Calif_pop</b>, <b>Calif_high_earners</b>, and <b>Calif_median_age</b>. The resultant GeoDataFrame will name <b>Calif_stats</b>, and have 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".

In [15]:
DF_comb = Calif_pop.join([Calif_high_earners, Calif_median_age]).reset_index()
Calif_stats = Calif_counties.merge(DF_comb, on='GEOID')
Calif_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.6556 39.69357, -120.65554 39.69...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5


Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns. I'll create a <b>"density"</b> column with the population density.

In [16]:
Calif_stats["density"] = Calif_stats["population"] / Calif_stats["area_sqkm"]

We need one more GeoDataFrame <b>sel_counties</b> that contains a subset of the rows (and all of the columns) from the Calif_stats GeoDataFrame. In particular, counties where:

-there are at least 100,000 households making $150,000 per year,

-the median age is less than 38.5, and

-the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:

-there are at least 500,000 households making $150,000 per year,

-the median age is less than 35.5, or

-the density of inhabitants is at least 1400 (per square kilometer).

In [17]:
sel_counties = Calif_stats[
        ((Calif_stats['high_earners']>=100000)&(Calif_stats['median_age']<38.5)&(Calif_stats['density']>=285))
    &((Calif_stats['high_earners']>=500000)
    |(Calif_stats['median_age']<35.5)
    |(Calif_stats['density']>=1400))
    ]

When looking for the next Starbucks Reserve Roastery location, I'd like to consider all of the stores within the counties that I selected. So, how many stores are within the selected counties?

To prepare to answer this question, I create a GeoDataFrame <b>starbucks_gdf</b> with all of the starbucks locations.

In [18]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}
starbucks_gdf.head()

c:\Users\polin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.4 34.51)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76 34.16)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76 34.15)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75 33.87)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28 37.79)


So, how many stores are in the counties I selected?

In [20]:
num_stores = len(gpd.sjoin(sel_counties, starbucks_gdf))
num_stores

1043

I create a map that shows the locations of the stores that I identified in the previous question.

In [21]:
map = folium.Map(location=[37,-120], zoom_start=6)

sel_stores = gpd.sjoin(sel_counties, starbucks_gdf)
mc = folium.plugins.MarkerCluster()
for idx, row in sel_stores.iterrows():
    mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
map.add_child(mc)


map